In [1]:
from pkg import *
paths = path.PathManager(datasets=[1,2,3])
# print(paths.total_paths.peaks)
# print(paths.total_paths.water_background)

processor = process.Processor(paths=paths, datasets=[1,2,3])
# print(processor.water_backgrounds)

# focus on water_background dict
datasets_int = f.convert2int(processor.datasets)
# print(processor.datasets)
# print(datasets_int)

# returns dictionary of selected water_background paths
# print(processor.water_background_dict)

# print('\n', processor.dataset_dict)



['01', '02', '03']
['/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00004.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00010.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00078.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00068.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00011.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00079.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00069.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00001.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00005.h5', '/Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/img_6keV_clen01_00085.h5', 

### Changes to `path.py`
- `path.py` now takes in a list of ints, where each int corresponds to a dataset index. Inside of `path.select_datasets()`, this iterates through the list of ints and appends all paths to the corresponding datasets to the `datasets` list. This allows for the selection of multiple datasets at once.
- The instance of PathManager, `myPaths`, now has the attribute of `total_paths` where this is a namedtuple that contains all the paths to the dataset(s).

- This change makes it easier to select multiple datasets at once, and allows for the selection of datasets in a more flexible manner.


- `init_list()` function takes the string `dataset` argument that pads the integer and has been casted to a string, then uses the `get_()` functions for all the files in the directories we need, returns many lists. 

In [2]:
ints = f.convert2int(datasets=['01','02'])
print(ints)
strs = f.convert2str(datasets=[1,2])
print(strs)

[1, 2]
['01', '02']


In [35]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import subprocess
from glob import glob
from pkg import *

datasets = [1,2,3]

myPaths = path.PathManager(datasets=datasets)
myProcessor = process.Processor(paths=myPaths, datasets=datasets)
params = myProcessor.get_parameters()
print(params)
f.get_counts(paths=myPaths, datasets=datasets)

myDatasetManager = data.DatasetManager(paths=myPaths, datasets=datasets)

# peak, label, overlay are valid types
parameters_4_datasets = myProcessor.get_parameters()
print(parameters_4_datasets)

train_loader, test_loader = f.prepare(data_manager=myDatasetManager, batch_size=10)

['01', '02', '03']
Water backgrounds initialized: 3
{'01': {'clen': 0.15, 'photon_energy': 6000}, '02': {'clen': 0.15, 'photon_energy': 7000}, '03': {'clen': 0.15, 'photon_energy': 8000}}
Paths refreshed for dataset ['01', '02', '03'].
Directory: peaks/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: labels/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: peaks_water_overlay/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: peaks/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: labels/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: peaks_water_overlay/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: peaks/03
	Total files: 22
	Normal images: 15
	Empty images: 7
Directory: labels/03
	Total files: 22
	Normal images: 15
	Empty images: 7
Directory: peaks_water_overlay/03
	Total files: 22
	Normal images: 15
	Empty images: 7
Actual percentage of empty images: 32.38095238095238% acros

In [2]:
f.check_attributes()

AttributeError: module 'pkg.functions' has no attribute 'check_attributes'

In [2]:
f.get_counts(paths=myPaths, datasets=datasets)
# %run process_directory.py ../../images  --percent_empty 0.5 
# f.get_counts(paths=myPaths, datasets=datasets)

Paths refreshed for dataset ['01', '02', '03'].
Directory: peaks/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: labels/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: peaks_water_overlay/01
	Total files: 58
	Normal images: 39
	Empty images: 19
Directory: peaks/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: labels/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: peaks_water_overlay/02
	Total files: 25
	Normal images: 17
	Empty images: 8
Directory: peaks/03
	Total files: 22
	Normal images: 15
	Empty images: 7
Directory: labels/03
	Total files: 22
	Normal images: 15
	Empty images: 7
Directory: peaks_water_overlay/03
	Total files: 22
	Normal images: 15
	Empty images: 7


In [47]:
from typing import List, Dict, Any
from pkg.functions import convert2int, convert2str, get_params

datasets = [1,2,3]
datasets = f.convert2str(datasets)
params = f.get_params(datasets=datasets)
print(params)
# f.check_attributes(paths=myPaths, datasets=datasets, dir_type='peak')

def check_attributes(paths: path.PathManager, datasets: List[str], dir_type: str) -> None:
    conform = True
    params = get_params(datasets=datasets)
    for dataset in datasets:
        files = paths.fetch_paths_by_type(dataset=dataset, dir_type=dir_type)
        exp_clen, exp_photon_energy = params.get(dataset)['clen'], params.get(dataset)['photon_energy']
        for path in files:
            attributes = f.retrieve_attributes(path)
            act_clen, act_photon_energy = attributes['clen'], attributes['photon_energy']
            if act_clen != exp_clen or act_photon_energy != exp_photon_energy:
                conform = False
                print(f'Error: {path} does not match expected attributes')
                print(f'Expected: clen={exp_clen}, photon_energy={exp_photon_energy}')
                print(f'Actual: clen={act_clen}, photon_energy={act_photon_energy}')
        return conform

conform = check_attributes(paths=myPaths, datasets=datasets, dir_type='peak')
print(conform)


{'01': {'clen': 1.5, 'photon_energy': 6000}, '02': {'clen': 1.5, 'photon_energy': 7000}, '03': {'clen': 1.5, 'photon_energy': 8000}}
True


In [6]:
model1 = m.BasicCNN1()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = {
    'num_epochs': 10,
    'num_classes': 2,
    'batch_size': train_loader.batch_size,
    'test_size': len(train_loader.dataset),
    'test_size': len(test_loader.dataset),
    'criterion': criterion,
    'optimizer': optimizer,
    'device': device,
    'model': model1,
    }

# arguments: self, model, loader: list, criterion, optimizer, device, cfg: dict
t = train_eval.TrainTestModels(model=model1, loader=[train_loader, test_loader], criterion=criterion, optimizer=optimizer, device=device, cfg=cfg)
t.epoch_loop()

Model testing and validating: BasicCNN1
-- epoch 0
12
11
9
27
17
26
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 1
17
18
13
14
17
23
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 2
23
25
14
12
10
18
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 3
32
11
8
22
12
17
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 4
17
16
19
22
9
19
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 5
11
18
16
20
19
18
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 6
20
16
22
5
20
19
Train loss: 0.1879569947719574
Train accuracy: 1.0
Test loss: 0.06265233159065246
Test accuracy: 1.0
-- epoch 7
21
12
20
8
30
11
Train loss: 0.187956994771

In [ ]:
test_peaks = myPaths.get_peak_image_paths(dataset=dataset)[0]
test_water = myPaths.get_peaks_water_overlay_image_paths(dataset=dataset)[0]
test_overlay = myPaths.get_peaks_water_overlay_image_paths(dataset=dataset)[0]
param1, param2 = (0.5, 9000), (0.5, 9000, True)
f.assign_attributes(file_path=test_peaks, params=param1)

Attributes ['params'] assigned to /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/01/empty_peak_01_00020.h5
